In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.window import Window

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
spark = SparkSession.builder.appName('ReadData').getOrCreate()
sc = spark.sparkContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fd60219c-c362-43b3-9116-862a3282722f;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.4.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok;1.16.8 in central
	found com.google.cloud#google-cloud-storage;2.16.0 in central
	found com.google.guava#guava;31.1-jre in centra

In [4]:
spark = SparkSession.builder \
    .appName("Read All CSV Files in a Folder") \
    .getOrCreate()

In [5]:
bucket_name = 'msca-bdp-student-gcs'
folder_path = 'group_1/unzipped_files'

In [6]:
input_path = f"gs://msca-bdp-student-gcs/group_1/unzipped_files/*.csv"
df = spark.read.csv(input_path, header=True, inferSchema=True)

In [60]:
print(df.printSchema())
df.show(5)

root
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- DOT_ID_Reporting_Airline: integer (nullable = true)
 |-- IATA_CODE_Reporting_Airline: string (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- OriginAirportSeqID: integer (nullable = true)
 |-- OriginCityMarketID: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- OriginState: string (nullable = true)
 |-- OriginStateFips: integer (nullable = true)
 |-- OriginStateName: string (nullable = true)
 |-- OriginWac: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DestAirportSeqID: integer (nullable = true)
 |-- DestCityMark

+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-------------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+
|Year|Quarter|Month|DayofMonth|DayOfWeek|FlightDate|Reporting_Airline|

In [62]:
from pyspark.sql.functions import col, to_date

df = df.withColumn('Cancelled', col('Cancelled').cast('int'))
df = df.withColumn('DepDel15', col('DepDel15').cast('int'))
df = df.withColumn('ArrDel15', col('ArrDel15').cast('int'))
df = df.withColumn("FlightDate", to_date(col("FlightDate"), "yyyy-MM-dd"))

In [63]:
df = df.drop("Tail_Number","Div4TailNum","Div5Airport","Div5Airport",'Div1WheelsOff',
 'Div1TailNum',
 'Div2Airport',
 'Div2AirportID',
 'Div2AirportSeqID',
 'Div2WheelsOn',
 'Div2TotalGTime',
 'Div2LongestGTime',
 'Div2WheelsOff',
 'Div2TailNum',
 'Div3Airport',
 'Div3AirportID',
 'Div3AirportSeqID',
 'Div3WheelsOn',
 'Div3TotalGTime',
 'Div3LongestGTime',
 'Div3WheelsOff',
 'Div3TailNum',
 'Div4Airport',
 'Div4AirportID',
 'Div4AirportSeqID',
 'Div4WheelsOn',
 'Div4TotalGTime',
 'Div4LongestGTime',
 'Div4WheelsOff',
 'Div4TailNum',
 'Div5Airport',
 'Div5AirportID',
 'Div5AirportSeqID',
 'Div5WheelsOn',
 'Div5TotalGTime',
 'Div5LongestGTime',
 'Div5WheelsOff',
 'Div5TailNum',
'Div1LongestGTime',
'Div1TotalGTime',
'Div1WheelsOn',
'Div1AirportSeqID',
 '_c109')

In [64]:
df.describe().toPandas()

23/08/16 00:14:12 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 0.0 in stage 49.1 (TID 7178) (hub-hub-msca-bdp-dphub-student-sanchiarora-sw-j1k1.c.msca-bdp-student-ap.internal executor 268): FetchFailed(BlockManagerId(239, hub-hub-msca-bdp-dphub-student-sanchiarora-sw-7109.c.msca-bdp-student-ap.internal, 7337, None), shuffleId=5, mapIndex=20, mapId=5553, reduceId=0, message=
org.apache.spark.shuffle.FetchFailedException: Failure while fetching StreamChunkId[streamId=390991035000,chunkIndex=0]: java.lang.RuntimeException: Executor is not registered (appId=application_1692138744335_0001, execId=239)
	at org.apache.spark.network.shuffle.ExternalShuffleBlockResolver.getContinuousBlocksData(ExternalShuffleBlockResolver.java:203)
	at org.apache.spark.network.shuffle.ExternalShuffleBlockResolver.getBlockData(ExternalShuffleBlockResolver.java:185)
	at org.apache.spark.network.shuffle.ExternalBlockHandler$ShuffleManagedBufferIterator.next(ExternalBlockHandler.java:533)
	at org.apach

KeyboardInterrupt: 

In [65]:
from pyspark.sql.functions import col

for column in df.columns:
   df_new = df.withColumn(column, col(column).cast("string"))

In [66]:
df_new.show(12)

+----+-------+-----+----------+---------+----------+-----------------+------------------------+---------------------------+-------------------------------+---------------+------------------+------------------+------+--------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+----+-------------------+---------+-------------+-------------+-------+----------+-------+--------+---------------+--------+--------------------+----------+-------+---------+--------+------+----------+-------+--------+---------------+--------+------------------+----------+---------+----------------+--------+--------------+-----------------+-------+-------+--------+-------------+------------+------------+--------+-------------+-----------------+------------+-------------+---------------+------------------+--------------+--------------------+-----------+-----------+-----------+-------------+
|Year|Quarter|Month|DayofMonth|DayOfWeek|FlightDate|Reporting_Airline|

In [67]:
df_new.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAir

In [68]:
df_new.count()

210832937

In [69]:
import pandas as pd
pd.set_option('display.max_columns',None)

In [48]:
df_new.limit(5).toPandas()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID
0,1988,1,1,8,5,1988-01-08,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1532.0,7.0,7.0,0.0,0.0,1500-1559,NaN,None,None,NaN,1545,1555.0,10.0,10.0,0.0,0.0,1500-1559,0,None,0.0,20.0,23.0,NaN,1.0,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,1988,1,1,9,6,1988-01-09,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1535.0,-10.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,13.0,NaN,1.0,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,1988,1,1,10,7,1988-01-10,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1534.0,-11.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,12.0,NaN,1.0,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,1988,1,1,11,1,1988-01-11,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,NaN,NaN,NaN,NaN,NaN,1500-1559,NaN,None,None,NaN,1545,NaN,NaN,NaN,NaN,NaN,1500-1559,1,None,0.0,20.0,NaN,NaN,1.0,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
4,1988,1,1,12,2,1988-01-12,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1524.0,-1.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1540.0,-5.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,16.0,NaN,1.0,32.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None


In [70]:
df_new = df_new.drop("CarrierDelay","WeatherDelay","NASDelay","SecurityDelay","LateAircraftDelay","FirstDepTime","TotalAddGTime","LongestAddGTime","DivAirportLandings","DivReachedDest","DivActualElapsedTime","DivArrDelay","DivDistance","Div1Airport","Div1AirportID")

In [50]:
df_new.limit(5).toPandas()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup
0,1988,1,1,8,5,1988-01-08,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1532.0,7.0,7.0,0.0,0.0,1500-1559,NaN,None,None,NaN,1545,1555.0,10.0,10.0,0.0,0.0,1500-1559,0,None,0.0,20.0,23.0,NaN,1.0,32.0,1
1,1988,1,1,9,6,1988-01-09,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1535.0,-10.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,13.0,NaN,1.0,32.0,1
2,1988,1,1,10,7,1988-01-10,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1534.0,-11.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,12.0,NaN,1.0,32.0,1
3,1988,1,1,11,1,1988-01-11,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,NaN,NaN,NaN,NaN,NaN,1500-1559,NaN,None,None,NaN,1545,NaN,NaN,NaN,NaN,NaN,1500-1559,1,None,0.0,20.0,NaN,NaN,1.0,32.0,1
4,1988,1,1,12,2,1988-01-12,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1524.0,-1.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1540.0,-5.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,16.0,NaN,1.0,32.0,1


### Join Airline Name Data Soure
##### Read CSV of Carrier Names

In [71]:
airline_df = spark.read.csv("gs://msca-bdp-student-gcs/group_1/L_CARRIER_HISTORY.csv", inferSchema=True, header=True)

In [72]:
airline_df.show(10)

+----+--------------------+
|Code|         Description|
+----+--------------------+
| 02Q|      Titan Airways |
| 04Q| Tradewind Aviation |
| 05Q|Comlux Aviation, AG |
| 06Q|Master Top Linhas...|
| 07Q|Flair Airlines Ltd. |
| 09Q|     Swift Air, LLC |
| 09Q|Swift Air, LLC d/...|
| 0BQ|                DCA |
| 0CQ|ACM AIR CHARTER G...|
| 0FQ|Maine Aviation Ai...|
+----+--------------------+
only showing top 10 rows



#### Join Data Soure to DF, make new airline DF

In [73]:
airline_df_new = df_new.join(airline_df, df_new["IATA_CODE_Reporting_Airline"] == airline_df["code"], how="inner")
airline_df_new.limit(5).toPandas()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,Code,Description
0,1988,1,1,8,5,1988-01-08,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1532.0,7.0,7.0,0.0,0.0,1500-1559,NaN,None,None,NaN,1545,1555.0,10.0,10.0,0.0,0.0,1500-1559,0,None,0.0,20.0,23.0,NaN,1.0,32.0,1,PI,Piedmont Aviation Inc.
1,1988,1,1,9,6,1988-01-09,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1535.0,-10.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,13.0,NaN,1.0,32.0,1,PI,Piedmont Aviation Inc.
2,1988,1,1,10,7,1988-01-10,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1522.0,-3.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1534.0,-11.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,12.0,NaN,1.0,32.0,1,PI,Piedmont Aviation Inc.
3,1988,1,1,11,1,1988-01-11,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,NaN,NaN,NaN,NaN,NaN,1500-1559,NaN,None,None,NaN,1545,NaN,NaN,NaN,NaN,NaN,1500-1559,1,None,0.0,20.0,NaN,NaN,1.0,32.0,1,PI,Piedmont Aviation Inc.
4,1988,1,1,12,2,1988-01-12,PI,19822,PI,930,10577,1057701,30577,BGM,"Binghamton, NY",NY,36,New York,22,12397,1239701,32397,ITH,"Ithaca/Cortland, NY",NY,36,New York,22,1525,1524.0,-1.0,0.0,0.0,-1.0,1500-1559,NaN,None,None,NaN,1545,1540.0,-5.0,0.0,0.0,-1.0,1500-1559,0,None,0.0,20.0,16.0,NaN,1.0,32.0,1,PI,Piedmont Aviation Inc.


In [74]:
airline_df_new.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'Code',
 'Description']

#### Let's look at total departure delays by airline over time 

In [79]:
#result = airline_df_new.groupBy("IATA_CODE_Reporting_Airline").agg(F.sum("DepDelay").alias("Total_DepDelay"))
#result.show()
from pyspark.sql import functions as F


In [81]:
df_not_cancelled = airline_df_new.filter(airline_df_new["Cancelled"] == 0)

result = df_not_cancelled.select('Year', 'ArrDel15', 'DepDel15') \
    .groupby('Year') \
    .agg(F.sum('ArrDel15').alias("total_arrival_delay"),
         F.sum('DepDel15').alias("total_departure_delay")) \
    .orderBy('Year')

result.show(5)

+----+-------------------+---------------------+
|Year|total_arrival_delay|total_departure_delay|
+----+-------------------+---------------------+
|1988|            1202126|               831516|
|1989|            1433465|              1041419|
|1990|            1292084|               887568|
|1991|            1051027|               721292|
|1992|            1039654|               707480|
+----+-------------------+---------------------+
only showing top 5 rows



In [83]:
df_delay_month = df_not_cancelled.select('Month', 'ArrDel15', 'DepDel15').groupby('Month') \
.agg(F.sum('ArrDel15').alias("total_arrival_delay"), 
     F.sum('DepDel15').alias("total_departure_delay"), 
     F.count("*").alias("flight_count")) \
.withColumn('percent_arr_delayed', col('total_arrival_delay') / col('flight_count') * 100) \
.withColumn('percent_dep_delayed', col('total_departure_delay') / col('flight_count') * 100) \
.orderBy('Month')

df_delay_month.show(5)

+-----+-------------------+---------------------+------------+-------------------+-------------------+
|Month|total_arrival_delay|total_departure_delay|flight_count|percent_arr_delayed|percent_dep_delayed|
+-----+-------------------+---------------------+------------+-------------------+-------------------+
|    1|            5069024|              4335340|    23563257| 21.512408068205513|  18.39872985300801|
|    2|            4586890|              3901410|    21739988|  21.09886169210397|   17.9457780749465|
|    3|            4918145|              4289598|    24727766| 19.889160225796378|  17.34729291760525|
|    4|            4249609|              3639439|    23567098| 18.031957095438734| 15.442881427318714|
|    5|            4339225|              3699500|    23477228| 18.482697361034276| 15.757822857110728|
+-----+-------------------+---------------------+------------+-------------------+-------------------+
only showing top 5 rows



#### Now, let's further break down by year and add airline name

In [85]:
result_byyear = df_not_cancelled.select('Year',"IATA_CODE_Reporting_Airline", "Description", 'ArrDel15', 'DepDel15') \
    .groupby('Year') \
    .agg(F.sum('ArrDel15').alias("total_arrival_delay"),
         F.sum('DepDel15').alias("total_departure_delay")) \
    .orderBy('Year')
result_byyear.show(10)

+----+-------------------+---------------------+
|Year|total_arrival_delay|total_departure_delay|
+----+-------------------+---------------------+
|1988|            1202126|               831516|
|1989|            1433465|              1041419|
|1990|            1292084|               887568|
|1991|            1051027|               721292|
|1992|            1039654|               707480|
|1993|            1026368|               730803|
|1994|            1086479|               835312|
|1995|            1244404|               993301|
|1996|            1451869|              1163917|
|1997|            1258180|               980295|
+----+-------------------+---------------------+
only showing top 10 rows



In [86]:
df_delay_alltime = df_not_cancelled.select('IATA_CODE_Reporting_Airline', 'Description', 'ArrDel15', 'DepDel15') \
.groupby('IATA_CODE_Reporting_Airline', 'Description') \
.agg(F.sum('ArrDel15').alias("total_arrival_delay"), 
     F.sum('DepDel15').alias("total_departure_delay"), 
     F.count("*").alias("flight_count")) \
.withColumn('percent_arr_delayed', col('total_arrival_delay') / col('flight_count') * 100) \
.withColumn('percent_dep_delayed', col('total_departure_delay') / col('flight_count') * 100) \
.orderBy('IATA_CODE_Reporting_Airline', 'Description')

df_delay_alltime.show(25)

+---------------------------+--------------------+-------------------+---------------------+------------+-------------------+-------------------+
|IATA_CODE_Reporting_Airline|         Description|total_arrival_delay|total_departure_delay|flight_count|percent_arr_delayed|percent_dep_delayed|
+---------------------------+--------------------+-------------------+---------------------+------------+-------------------+-------------------+
|                         9E|  Endeavor Air Inc. |             410519|               354060|     2556754|  16.05625726996027| 13.848027616266561|
|                         9E|Pinnacle Airlines...|             410519|               354060|     2556754|  16.05625726996027| 13.848027616266561|
|                         AA|American Airlines...|            4875749|              3960206|    24438635| 19.950987442629263| 16.204693920098237|
|                         AL|    Skyway Airlines |              82148|                59273|      357672|  22.96741148314657

#### Compare delays by arrival time and airline

In [87]:
Arrivaldelay_by_alltime_ = df_not_cancelled.select('IATA_CODE_Reporting_Airline', 'Description', 'ArrDel15', 'DepDel15') \
.groupby('IATA_CODE_Reporting_Airline', 'Description') \
.agg(F.sum('DepDel15').alias("total_departure_delay"), 
     F.count("*").alias("flight_count")) \
.withColumn('percent_dep_delayed', col('total_departure_delay') / col('flight_count') * 100) \
.orderBy('IATA_CODE_Reporting_Airline', 'Description')

Arrivaldelay_by_alltime_.show(25)

+---------------------------+--------------------+---------------------+------------+-------------------+
|IATA_CODE_Reporting_Airline|         Description|total_departure_delay|flight_count|percent_dep_delayed|
+---------------------------+--------------------+---------------------+------------+-------------------+
|                         9E|  Endeavor Air Inc. |               354060|     2556754| 13.848027616266561|
|                         9E|Pinnacle Airlines...|               354060|     2556754| 13.848027616266561|
|                         AA|American Airlines...|              3960206|    24438635| 16.204693920098237|
|                         AL|    Skyway Airlines |                59273|      357672| 16.571887092084367|
|                         AL|              USAir |                59273|      357672| 16.571887092084367|
|                         AQ|Aloha Airlines Inc. |                13268|      151544|  8.755213007443382|
|                         AS|Alaska Airlines I

In [88]:
Arrivaldelay_by_alltime_ = df_not_cancelled.select('IATA_CODE_Reporting_Airline', 'Description', 'DepDel15') \
.groupby('IATA_CODE_Reporting_Airline', 'Description') \
.agg(F.sum('DepDel15').alias("total_departure_delay"), 
     F.count("*").alias("flight_count")) \
.withColumn('percent_dep_delayed', col('total_departure_delay') / col('flight_count') * 100) \
.orderBy(col('percent_dep_delayed').desc(), 'IATA_CODE_Reporting_Airline', 'Description')

Arrivaldelay_by_alltime_.show(25)

+---------------------------+--------------------+---------------------+------------+-------------------+
|IATA_CODE_Reporting_Airline|         Description|total_departure_delay|flight_count|percent_dep_delayed|
+---------------------------+--------------------+---------------------+------------+-------------------+
|                         G4|      Allegiant Air |               127157|      547424|  23.22824720874496|
|                         B6|    JetBlue Airways |               977851|     4229536| 23.119580965855356|
|                         EV|Atlantic Southeas...|              1396704|     6416359|  21.76785931086462|
|                         EV|ExpressJet Airlin...|              1396704|     6416359|  21.76785931086462|
|                         EV|ExpressJet Airlin...|              1396704|     6416359|  21.76785931086462|
|                         EV|ExpressJet Airlin...|              1396704|     6416359|  21.76785931086462|
|                         F9|Frontier Airlines

#### Some intereting trends, while American Airlines has several departure delays, it seems they make up time and do not have the most arrival delays

#### Let's Plot Arrival and Departure Delay over time to see if there are any trends by Year

In [26]:
pdf=airline_df_new.limit(1000).toPandas()

#### Yearly delays by Arrival and Departure

##### Avg arrival delay by year

In [94]:
airline_df_new.createOrReplaceTempView("airline_data")

query = """
SELECT Year, AVG(ArrDel15) * 100 as AveragePercentArrDelay
FROM airline_data
GROUP BY Year
ORDER BY Year
"""

yearly_arrdelays_sql = spark.sql(query)

In [93]:
airline_df_new.createOrReplaceTempView("airline_data")

query = """
WITH YearlyDelays AS (
    SELECT 
        Year,
        SUM(ArrDel15) as TotalArrivalDelay, 
        COUNT(*) as FlightCount
    FROM airline_data
    GROUP BY Year
)

SELECT 
    Year,
    (TotalArrivalDelay / FlightCount) * 100 as AveragePercentArrDelay
FROM YearlyDelays
ORDER BY Year
"""

yearly_avg_arrdelays = spark.sql(query)
yearly_avg_arrdelays.show()


+----+----------------------+
|Year|AveragePercentArrDelay|
+----+----------------------+
|1988|    19.292662607500784|
|1989|    22.869531838803116|
|1990|    19.038393759113376|
|1991|     16.21783307849924|
|1992|      16.7015078609156|
|1993|    16.693215518668715|
|1994|    17.256219228262818|
|1995|    19.508672740401604|
|1996|     22.81846696045212|
|1997|    19.644847611048945|
|1998|     19.67639072301609|
|1999|    21.014356233921806|
|2000|    23.720629449862827|
|2001|    18.723605138777057|
|2002|     16.57416316486095|
|2003|    17.522522504563902|
|2004|     20.37767325000461|
|2005|      20.9399686952173|
|2006|     23.62748455944165|
|2007|    25.210729532363967|
+----+----------------------+
only showing top 20 rows



In [28]:
query = """
SELECT Year, MAX(ArrDel) as MaxArrDelay
FROM airline_data
GROUP BY Year
ORDER BY MaxArrDelay DESC
LIMIT 1
"""

max_yearly_delays_sql = spark.sql(query)
max_yearly_delays_sql.show()

+----+-----------+
|Year|MaxArrDelay|
+----+-----------+
|2023|     4405.0|
+----+-----------+



In [29]:
max_year = 2023

windowSpec = Window.partitionBy("Year", "Month").orderBy(F.col("ArrDelay").desc())

ranked_df = (
    airline_df_new.withColumn("rank", F.row_number().over(windowSpec))
    .filter(F.col("Year") == max_year)
)

max_monthly_delays_2023 = (
    ranked_df.filter(F.col("rank") == 1)
    .select("Month", "ArrDelay", "Reporting_Airline", "Description")
    .withColumnRenamed("ArrDelay", "MaxArrDelayPerMonth")
    .withColumnRenamed("Reporting_Airline", "AirlineWithMaxDelay")
    .orderBy("Month")
)

max_monthly_delays_2023.show()


+-----+-------------------+-------------------+--------------------+
|Month|MaxArrDelayPerMonth|AirlineWithMaxDelay|         Description|
+-----+-------------------+-------------------+--------------------+
|    1|             3063.0|                 AA|American Airlines...|
|    2|             2949.0|                 AA|American Airlines...|
|    3|             4405.0|                 AA|American Airlines...|
|    4|             2916.0|                 AA|American Airlines...|
+-----+-------------------+-------------------+--------------------+



In [30]:
from pyspark.sql.window import Window

windowSpec = Window.partitionBy("Year", "Month").orderBy(F.col("ArrDelay").desc())

ranked_df = (
    airline_df_new.withColumn("rank", F.row_number().over(windowSpec))
    .filter(F.col("Year") == 2023)
)

top_delays_2023 = (
    ranked_df.filter(F.col("rank") == 1)
    .select("Month", "Reporting_Airline", "Description", "ArrDelay")
    .withColumnRenamed("ArrDelay", "MaxArrDelayPerMonth")
    .orderBy("Month")
)

top_delays_2023.show()

+-----+-----------------+--------------------+-------------------+
|Month|Reporting_Airline|         Description|MaxArrDelayPerMonth|
+-----+-----------------+--------------------+-------------------+
|    1|               AA|American Airlines...|             3063.0|
|    2|               AA|American Airlines...|             2949.0|
|    3|               AA|American Airlines...|             4405.0|
|    4|               AA|American Airlines...|             2916.0|
+-----+-----------------+--------------------+-------------------+



##### Avg departure delay by year

In [95]:
airline_df_new.createOrReplaceTempView("airline_data")

query = """
WITH YearlyDelays AS (
    SELECT 
        Year,
        SUM(DepDel15) as TotalDepDelay, 
        COUNT(*) as FlightCount
    FROM airline_data
    GROUP BY Year
)

SELECT 
    Year,
    (TotalDepDelay / FlightCount) * 100 as AveragePercentDepDelay
FROM YearlyDelays
ORDER BY Year
"""

yearly_avg_depdelays = spark.sql(query)
yearly_avg_depdelays.show()



+----+----------------------+
|Year|AveragePercentDepDelay|
+----+----------------------+
|1988|    13.344822124085681|
|1989|    16.614821413870935|
|1990|    13.077995758780961|
|1991|    11.129869410449848|
|1992|    11.365303054132019|
|1993|    11.886040855414096|
|1994|    13.267009298843945|
|1995|    15.572100492857347|
|1996|    18.292836067998252|
|1997|    15.306034024442624|
|1998|    16.050955935207647|
|1999|    17.176085707833398|
|2000|    19.663650306427925|
|2001|    16.200675500327723|
|2002|    13.694900683491843|
|2003|    13.918134378728917|
|2004|    17.127644595755108|
|2005|     18.05809507498333|
|2006|    20.779355197459584|
|2007|    22.198416958864186|
+----+----------------------+
only showing top 20 rows



In [32]:
query = """
SELECT Year, MAX(DepDelay) as MaxArrDelay
FROM airline_data
GROUP BY Year
ORDER BY MaxArrDelay DESC
LIMIT 1
"""

max_yearly_depdelays_sql = spark.sql(query)
max_yearly_depdelays_sql.show()

+----+-----------+
|Year|MaxArrDelay|
+----+-----------+
|2023|     4413.0|
+----+-----------+



In [33]:
max_dep_year = (
    airline_df_new.groupBy("Year")
    .agg(F.max("DepDelay").alias("MaxDepDelay"))
    .orderBy("MaxDepDelay", ascending=False)
    .limit(1)
    .collect()[0]["Year"]
)

max_monthly_depdelays = (
    airline_df_new.filter(airline_df_new["Year"] == max_dep_year)
    .groupBy("Month")
    .agg(F.max("DepDelay").alias("MaxDepDelayPerMonth"), 
         F.first("Reporting_Airline").alias("AirlineWithMaxDelay"))
    .orderBy("Month")
)

max_monthly_depdelays.show()

+-----+-------------------+-------------------+
|Month|MaxDepDelayPerMonth|AirlineWithMaxDelay|
+-----+-------------------+-------------------+
|    1|             3024.0|                 9E|
|    2|             2947.0|                 9E|
|    3|             4413.0|                 WN|
|    4|             2935.0|                 WN|
+-----+-------------------+-------------------+



#### Delays by Airline 

In [34]:
from pyspark.sql import functions as F

delayed_flights = airline_df_new.filter((F.col("ArrDelay") > 0) | (F.col("DepDelay") > 0))

total_delays_per_airline = (
    delayed_flights.groupBy("IATA_CODE_Reporting_Airline")
    .agg(
        F.count(F.when(F.col("ArrDelay") > 0, 1)).alias("TotalArrDelays"),
        F.count(F.when(F.col("DepDelay") > 0, 1)).alias("TotalDepDelays"),
    )
    .withColumn("TotalDelays", F.col("TotalArrDelays") + F.col("TotalDepDelays"))
    .orderBy("TotalDelays", ascending=False)
)

total_delays_per_airline.show()


+---------------------------+--------------+--------------+-----------+
|IATA_CODE_Reporting_Airline|TotalArrDelays|TotalDepDelays|TotalDelays|
+---------------------------+--------------+--------------+-----------+
|                         US|      15022012|      14536358|   29558370|
|                         WN|      13210356|      14645626|   27855982|
|                         DL|      12096899|      11515119|   23612018|
|                         AA|      10354744|       9526244|   19880988|
|                         EV|      10420428|       9108372|   19528800|
|                         MQ|       9833367|       8153172|   17986539|
|                         UA|       8595934|       8605137|   17201071|
|                         OO|       4447949|       3599340|    8047289|
|                         NW|       4593909|       3251818|    7845727|
|                         CO|       4067676|       3384886|    7452562|
|                         TW|       3551046|       2593120|    6

#### Let's make a new DF that looks at only data from 2000-2023

In [38]:
airline_df_new.columns

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Reporting_Airline',
 'DOT_ID_Reporting_Airline',
 'IATA_CODE_Reporting_Airline',
 'Flight_Number_Reporting_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'Code',
 'Description']

#### Consolidate Columns for GraphFrames

In [99]:
selected_columns = [
    'Year', 'Month', 'DayofMonth', 'ArrDelay', 'DepDelay','ArrDel15', 'DepDel15', 
    'Reporting_Airline', 'Origin', 'OriginCityName', 'FlightDate', 'Flight_Number_Reporting_Airline','DestCityName','Distance','DepTime','Description','IATA_CODE_Reporting_Airline']


In [100]:
Deepdive_df = airline_df_new.filter(
    (F.col('Year') >= 2000) & (F.col('Year') <= 2023)
).select(*selected_columns)

Deepdive_df.show(5)

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+
|2000|    1|         1|    null|    63.0|    null|       1|               DL|   ATL|   Atlanta, GA|2000-01-01|                           1408|Allentown/Bethleh...|   692.0|   1903|Delta Air Lines I...|                         DL|
|2000|    1|         2|    -1.0|    -1.0|       0|       0|               DL|   

In [101]:
Deepdive_df.printSchema()

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- ArrDelay: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrDel15: integer (nullable = true)
 |-- DepDel15: integer (nullable = true)
 |-- Reporting_Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- OriginCityName: string (nullable = true)
 |-- FlightDate: date (nullable = true)
 |-- Flight_Number_Reporting_Airline: integer (nullable = true)
 |-- DestCityName: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- IATA_CODE_Reporting_Airline: string (nullable = true)



In [116]:
Deepdive_df.show(1)

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+
|2000|    1|         1|    null|    63.0|    null|       1|               DL|   ATL|   Atlanta, GA|2000-01-01|                           1408|Allentown/Bethleh...|   692.0|   1903|Delta Air Lines I...|                         DL|
+----+-----+----------+--------+--------+--------+--------+-----------------+---

### Create airports as verticles and flights as edges

In [123]:
airports = Deepdive_df.withColumn("id",Deepdive_df["Origin"])

flights = Deepdive_df.withColumn("src", Deepdive_df["Origin"])
flights = flights.withColumn("dst", Deepdive_df["DestCityName"])

In [124]:
airports.show(4)

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline| id|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+
|2000|    1|         1|    null|    63.0|    null|       1|               DL|   ATL|   Atlanta, GA|2000-01-01|                           1408|Allentown/Bethleh...|   692.0|   1903|Delta Air Lines I...|                         DL|ATL|
|2000|    1|         2|    -1.0|    -1.0|       0|       0|     

In [125]:
from graphframes import *

In [126]:
graph = GraphFrame(airports, flights)

In [127]:
graph.vertices.show(5)

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline| id|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+
|2000|    1|         1|    null|    63.0|    null|       1|               DL|   ATL|   Atlanta, GA|2000-01-01|                           1408|Allentown/Bethleh...|   692.0|   1903|Delta Air Lines I...|                         DL|ATL|
|2000|    1|         2|    -1.0|    -1.0|       0|       0|     

In [107]:
graph.edges.show(5)

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+--------------------+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline|src|                 dst|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+--------------------+
|2000|    1|         1|    null|    63.0|    null|       1|               DL|   ATL|   Atlanta, GA|2000-01-01|                           1408|Allentown/Bethleh...|   692.0|   1903|Delta Air Lines I...|                         DL|ATL|Al

In [128]:
# # of Airports
graph.vertices.count()

209726424

In [ ]:
#Most connected flights
graph.degrees.orderBy("degree", ascending=False).show(50)

+--------------------+--------+
|                  id|  degree|
+--------------------+--------+
|                 ATL|14351145|
|         Atlanta, GA|14316515|
|         Chicago, IL|13826478|
|                 ORD|11737716|
|                 DFW|10320318|
|Dallas/Fort Worth...|10319487|
|        New York, NY| 7567823|
|         Houston, TX| 7565862|
|                 IAH| 6121748|
|      Washington, DC| 6056829|
|     Los Angeles, CA| 5643399|
|                 LAX| 5641836|
|                 CLT| 5569257|
|       Charlotte, NC| 5569217|
|                 DEN| 5302001|
|          Denver, CO| 5301384|
|         Phoenix, AZ| 4768313|
|                 PHX| 4737788|
|          Newark, NJ| 4636757|
|                 EWR| 4635481|
|                 LGA| 4519275|
|                 DTW| 4389425|
|         Detroit, MI| 4389376|
|          Boston, MA| 3966905|
|                 BOS| 3966619|
|       Las Vegas, NV| 3797756|
|                 LAS| 3797585|
|     Minneapolis, MN| 3616318|
|       

In [110]:
graph.edges.filter("Origin='SFO'").show()

+----+-----+----------+--------+--------+--------+--------+-----------------+------+-----------------+----------+-------------------------------+------------+--------+-------+--------------------+---------------------------+---+-----------+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|   OriginCityName|FlightDate|Flight_Number_Reporting_Airline|DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline|src|        dst|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+-----------------+----------+-------------------------------+------------+--------+-------+--------------------+---------------------------+---+-----------+
|2000|    1|        31|   -19.0|     6.0|       0|       0|               DL|   SFO|San Francisco, CA|2000-01-31|                             40| Atlanta, GA|  2139.0|    711|Delta Air Lines I...|                         DL|SFO|Atlanta, GA|
|2000|    1|         1|   -12.0|    

In [122]:
unique_values = Deepdive_df.select("Origin").distinct().collect()

# Convert the list of Rows to list 
unique_values_list = [row.Origin for row in unique_values]

print(unique_values_list)


['DLG', 'MSY', 'GEG', 'SNA', 'BUR', 'GTF', 'GRB', 'GRR', 'PSG', 'EUG', 'PVD', 'GSO', 'MYR', 'OAK', 'MSN', 'SCC', 'DCA', 'CID', 'MLU', 'WRG', 'HLN', 'LEX', 'FSM', 'FAR', 'MQT', 'BGM', 'LWB', 'GTR', 'FLO', 'LNY', 'IDA', 'COD', 'BTM', 'RDM', 'RFD', 'LWS', 'PSE', 'ISO', 'MTH', 'INL', 'JLN', 'ESC', 'CIU', 'PPG', 'DRT', 'JMS', 'UTM', 'IAG', 'IFP', 'PIB', 'EAR', 'SWO', 'PVU', 'ATY', 'DDC', 'FOD', 'ORF', 'SAV', 'CDV', 'TRI', 'CMH', 'MOB', 'PNS', 'IAH', 'HNL', 'SHV', 'CVG', 'SJC', 'BUF', 'LGA', 'TLH', 'KTN', 'LIH', 'CRW', 'CAK', 'ERI', 'TOL', 'EVV', 'TYR', 'ACT', 'GST', 'SCE', 'ADK', 'CHO', 'MCN', 'MOD', 'CEC', 'CDC', 'CWA', 'MWH', 'SPN', 'GCK', 'DIK', 'OGS', 'LBF', 'SLN', 'CDB', 'VCT', 'FMN', 'YNG', 'ENV', 'HPN', 'AUS', 'SJU', 'AVL', 'LGB', 'DUT', 'SRQ', 'RNO', 'SBN', 'JAC', 'CHS', 'RSW', 'TUL', 'HRL', 'AMA', 'ISP', 'BOS', 'MAF', 'MLB', 'EWR', 'LAS', 'FAI', 'JAN', 'ITO', 'MLI', 'GFK', 'RST', 'BIS', 'MIB', 'BFL', 'EYW', 'RDD', 'ATW', 'GJT', 'IPL', 'OGD', 'DHN', 'LYH', 'TUP', 'TTN', 'BJI', 'GCC'

In [134]:
#### View Short Haul Flights, Distance less than 500 miles
graph.edges.filter("Distance <= 500").dropDuplicates(['src','dst']).orderBy('Distance', ascending=False).show()

+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+--------------------+
|Year|Month|DayofMonth|ArrDelay|DepDelay|ArrDel15|DepDel15|Reporting_Airline|Origin|      OriginCityName|FlightDate|Flight_Number_Reporting_Airline|        DestCityName|Distance|DepTime|         Description|IATA_CODE_Reporting_Airline|src|                 dst|
+----+-----+----------+--------+--------+--------+--------+-----------------+------+--------------------+----------+-------------------------------+--------------------+--------+-------+--------------------+---------------------------+---+--------------------+
|2000|    1|         1|    15.0|    41.0|       1|       1|               NW|   DTW|         Detroit, MI|2000-01-01|                            160|       Charlotte, NC|   500.0|   1431|Northwest Airline...|          

In [135]:
### View longest flights from Source to Destination Distance
graph.edges.groupBy("src", "dst").max("Distance").orderBy("max(Distance)", ascending =False).show(25)

+---+-----------------+-------------+
|src|              dst|max(Distance)|
+---+-----------------+-------------+
|SFO|         Guam, TT|       5812.0|
|GUM|San Francisco, CA|       5812.0|
|HNL|       Boston, MA|       5095.0|
|BOS|     Honolulu, HI|       5095.0|
|JFK|     Honolulu, HI|       4983.0|
|HNL|     New York, NY|       4983.0|
|HNL|       Newark, NJ|       4963.0|
|EWR|     Honolulu, HI|       4963.0|
|OGG|       Newark, NJ|       4904.0|
|EWR|      Kahului, HI|       4904.0|
|IAD|     Honolulu, HI|       4817.0|
|HNL|   Washington, DC|       4817.0|
|MCO|     Honolulu, HI|       4757.0|
|HNL|      Orlando, FL|       4757.0|
|HNL|    Charlotte, NC|       4678.0|
|CLT|     Honolulu, HI|       4678.0|
|ATL|     Honolulu, HI|       4502.0|
|HNL|      Atlanta, GA|       4502.0|
|DTW|     Honolulu, HI|       4475.0|
|HNL|      Detroit, MI|       4475.0|
|HNL|   Cincinnati, OH|       4433.0|
|CVG|     Honolulu, HI|       4433.0|
|OGG|      Atlanta, GA|       4431.0|
|ATL|      K

In [138]:
# Most frequent flight routes delays
#flightRouteCount = graph.edges.groupBy("src", "dst").avg("DepDelay15").orderBy("avg(DepDelay15)", ascending=False).show(5)


route_delays = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_delays"),  # Sum of delays
    F.count("*").alias("flight_count")  # Count of flights
)

#percentage of delays for each route.
route_delays = route_delays.withColumn(
    "percent_delayed",
    (F.col("total_delays") / F.col("flight_count")) * 100
)

# Order the routes by percentage delays 
route_delays.orderBy(F.col("percent_delayed").desc()).show(100)


+---+--------------------+------------+------------+---------------+
|src|                 dst|total_delays|flight_count|percent_delayed|
+---+--------------------+------------+------------+---------------+
|MKE|          Albany, NY|           1|           1|          100.0|
|LAN|    Grand Rapids, MI|           2|           2|          100.0|
|LIT|Greensboro/High P...|           3|           3|          100.0|
|OKC|Bloomington/Norma...|           1|           1|          100.0|
|CPR|     Idaho Falls, ID|           2|           2|          100.0|
|BUR|          Fresno, CA|           2|           2|          100.0|
|ALB|    Grand Rapids, MI|           1|           1|          100.0|
|GSO|    Jacksonville, FL|           4|           4|          100.0|
|RAP|       Green Bay, WI|           1|           1|          100.0|
|ORD|          Pueblo, CO|           1|           1|          100.0|
|ABE|      South Bend, IN|           1|           1|          100.0|
|CMH|    Grand Rapids, MI|        

In [142]:
# Filtering short haul flights
#short_haul_flights = graph.edges.filter("Distance <= 500")

#flightRouteCount = (short_haul_flights.groupBy("src", "dst")
#                                    .agg(F.count("*").alias("RouteCount"), 
#                                         F.avg("DepDelay").alias("AvgDepDelay"))
#                                    .orderBy("RouteCount", "AvgDepDelay", ascending=[False, False])
#                   )

#flightRouteCount.show(5)

graph.edges.groupBy("src", "dst").min("Distance").orderBy("min(Distance)", ascending =True).show(25)

+---+--------------------+-------------+
|src|                 dst|min(Distance)|
+---+--------------------+-------------+
|GEG|         Spokane, WA|          6.0|
|ILE|         Killeen, TX|          8.0|
|OAK|   San Francisco, CA|         11.0|
|LGA|        New York, NY|         11.0|
|SFO|         Oakland, CA|         11.0|
|JFK|        New York, NY|         11.0|
|DAL|Dallas/Fort Worth...|         12.0|
|ORD|         Chicago, IL|         16.0|
|EWR|        New York, NY|         17.0|
|BUR|     Los Angeles, CA|         18.0|
|LAX|         Burbank, CA|         18.0|
|JFK|          Newark, NJ|         21.0|
|MIA| Fort Lauderdale, FL|         21.0|
|FLL|           Miami, FL|         21.0|
|HPN|        New York, NY|         22.0|
|ORF|Newport News/Will...|         23.0|
|DCA|      Washington, DC|         24.0|
|IAD|      Washington, DC|         24.0|
|IAH|         Houston, TX|         24.0|
|HOU|         Houston, TX|         24.0|
|SBP|     Santa Maria, CA|         25.0|
|GRB|        App

In [141]:
### Let's look at flights with the longest distance from origin to destination by most often delayed --DEPARTURE DELAY
# maximum distance for each flight route.
long_distance = graph.edges.groupBy("src", "dst").agg(
    F.max("Distance").alias("long_distance")
)

# percentage delays and join with long routs 
route_delays = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_delays"),  # Sum delays
    F.count("*").alias("flight_count")  # Count flights
).withColumn(
    "percent_delayed",
    (F.col("total_delays") / F.col("flight_count")) * 100
).join(
    long_distance, on=["src", "dst"], how="inner"
)

#order
top_routes = route_delays.orderBy(
    F.col("percent_delayed").desc(), 
    F.col("long_distance").desc()
)

top_routes.show(50)


23/08/16 02:45:41 WARN org.apache.spark.deploy.yarn.YarnAllocator: Container from a bad node: container_1692138744335_0001_01_000594 on host: hub-hub-msca-bdp-dphub-student-sanchiarora-sw-xn8j.c.msca-bdp-student-ap.internal. Exit status: 143. Diagnostics: [2023-08-16 02:45:41.543]Container killed on request. Exit code is 143
[2023-08-16 02:45:41.543]Container exited with a non-zero exit code 143. 
[2023-08-16 02:45:41.552]Killed by external signal
.
23/08/16 02:45:41 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 580 for reason Container from a bad node: container_1692138744335_0001_01_000594 on host: hub-hub-msca-bdp-dphub-student-sanchiarora-sw-xn8j.c.msca-bdp-student-ap.internal. Exit status: 143. Diagnostics: [2023-08-16 02:45:41.543]Container killed on request. Exit code is 143
[2023-08-16 02:45:41.543]Container exited with a non-zero exit code 143. 
[2023-08-16 02:45:41.552]Killed by external signal
.
23/08

+---+--------------------+------------+------------+---------------+-------------+
|src|                 dst|total_delays|flight_count|percent_delayed|long_distance|
+---+--------------------+------------+------------+---------------+-------------+
|FLL|        San Jose, CA|           1|           1|          100.0|       2557.0|
|LGB|          Newark, NJ|           1|           1|          100.0|       2444.0|
|TKI|       Texarkana, AR|           4|           4|          100.0|       2429.0|
|BUR| Fort Lauderdale, FL|           1|           1|          100.0|       2341.0|
|OAK|       Charlotte, NC|           1|           1|          100.0|       2287.0|
|CLT|         Oakland, CA|           1|           1|          100.0|       2287.0|
|PIT|         Oakland, CA|           1|           1|          100.0|       2243.0|
|GSP|   San Francisco, CA|           1|           1|          100.0|       2233.0|
|SFO|           Greer, SC|           1|           1|          100.0|       2233.0|
|LGB

In [ ]:
### Let's look at flights with the longest distance from origin to destination by most often delayed --ARRIVAL DELAY
# maximum distance for each flight route.
long_distance = graph.edges.groupBy("src", "dst").agg(
    F.max("Distance").alias("long_distance")
)

# percentage delays and join with long routs 
route_delays = graph.edges.groupBy("src", "dst").agg(
    F.sum("ArrDel15").alias("total_delays"),  # Sum delays
    F.count("*").alias("flight_count")  # Count flights
).withColumn(
    "percent_delayed",
    (F.col("total_delays") / F.col("flight_count")) * 100
).join(
    long_distance, on=["src", "dst"], how="inner"
)

#order
top_routes = route_delays.orderBy(
    F.col("percent_delayed").desc(), 
    F.col("long_distance").desc()
)

top_routes.show(50)


+---+--------------------+------------+------------+---------------+-------------+
|src|                 dst|total_delays|flight_count|percent_delayed|long_distance|
+---+--------------------+------------+------------+---------------+-------------+
|BUR| Fort Lauderdale, FL|           1|           1|          100.0|       2341.0|
|GSP|   San Francisco, CA|           1|           1|          100.0|       2233.0|
|GSP|        San Jose, CA|           1|           1|          100.0|       2211.0|
|LGB|         Orlando, FL|           1|           1|          100.0|       2203.0|
|ORF|           Boise, ID|           1|           1|          100.0|       2144.0|
|BHM|   San Francisco, CA|           2|           2|          100.0|       2013.0|
|BHM|        San Jose, CA|           1|           1|          100.0|       1990.0|
|CVG|      Sacramento, CA|           1|           1|          100.0|       1977.0|
|SMF|      Cincinnati, OH|           1|           1|          100.0|       1977.0|
|SDF

In [145]:
### Let's look at flights with the shortest distance from origin to destination by most often delayed --ARRIVAL DELAY

# shortest distance for each flight route.
short_distance = graph.edges.groupBy("src", "dst").agg(
    F.min("Distance").alias("short_distance")
)

# percentage delays with short routs 
route_delays = graph.edges.groupBy("src", "dst").agg(
    F.sum("ArrDel15").alias("total_delays"),  # Sum delays
    F.count("*").alias("flight_count")  # Count flights
).withColumn(
    "percent_delayed",
    (F.col("total_delays") / F.col("flight_count")) * 100
).join(
    short_distance, on=["src", "dst"], how="inner"
)

# Order
top_routes = route_delays.orderBy(
    F.col("percent_delayed").desc(), 
    F.col("short_distance").asc()
)

top_routes.show(50)


+---+--------------------+------------+------------+---------------+--------------+
|src|                 dst|total_delays|flight_count|percent_delayed|short_distance|
+---+--------------------+------------+------------+---------------+--------------+
|LAX|         Burbank, CA|           2|           2|          100.0|          18.0|
|OAK|        San Jose, CA|           2|           2|          100.0|          29.0|
|DCA|       Baltimore, MD|           1|           1|          100.0|          30.0|
|HPN|Newburgh/Poughkee...|           1|           1|          100.0|          36.0|
|PUB|Colorado Springs, CO|           4|           4|          100.0|          37.0|
|PNS|      Valparaiso, FL|           4|           4|          100.0|          40.0|
|PBI| Fort Lauderdale, FL|           3|           3|          100.0|          42.0|
|ACT|         Killeen, TX|           1|           1|          100.0|          45.0|
|MLB|         Orlando, FL|           2|           2|          100.0|        

In [146]:
### Let's look at flights with the shortest distance from origin to destination by most often delayed --DEPARTURE DELAY

# shortest distance for each flight route.
short_distance = graph.edges.groupBy("src", "dst").agg(
    F.min("Distance").alias("short_distance")
)

# percentage delays with short routs 
route_delays = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_delays"),  # Sum delays
    F.count("*").alias("flight_count")  # Count flights
).withColumn(
    "percent_delayed",
    (F.col("total_delays") / F.col("flight_count")) * 100
).join(
    short_distance, on=["src", "dst"], how="inner"
)

# Order
top_routes = route_delays.orderBy(
    F.col("percent_delayed").desc(), 
    F.col("short_distance").asc()
)

top_routes.show(50)


+---+--------------------+------------+------------+---------------+--------------+
|src|                 dst|total_delays|flight_count|percent_delayed|short_distance|
+---+--------------------+------------+------------+---------------+--------------+
|ILE|         Killeen, TX|           1|           1|          100.0|           8.0|
|LAX|         Burbank, CA|           2|           2|          100.0|          18.0|
|GRB|        Appleton, WI|           1|           1|          100.0|          25.0|
|HRL|     Brownsville, TX|           4|           4|          100.0|          26.0|
|OAK|        San Jose, CA|           2|           2|          100.0|          29.0|
|ONT|      Long Beach, CA|           1|           1|          100.0|          36.0|
|HPN|Newburgh/Poughkee...|           1|           1|          100.0|          36.0|
|PNS|      Valparaiso, FL|           4|           4|          100.0|          40.0|
|CLE|           Akron, OH|           1|           1|          100.0|        

In [151]:
### Latly, we want to see route by most avg dep delay %
greatest_avgDelay_route = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_departure_delays"),  # Sum of departure delays
    F.sum("ArrDel15").alias("total_arrival_delays"),  # Sum of arrival delays
    F.count("*").alias("flight_count")  # Count of flights
).withColumn(
    "percent_departure_delayed",
    (F.col("total_departure_delays") / F.col("flight_count")) * 100
).withColumn(
    "percent_arrival_delayed",
    (F.col("total_arrival_delays") / F.col("flight_count")) * 100
).withColumn(
    "average_percent_delayed",
    (F.col("percent_departure_delayed") + F.col("percent_arrival_delayed")) / 2
).orderBy(
    "average_percent_delayed", ascending=False
).limit(10)

greatest_avgDelay_route.show(10)


+---+--------------------+----------------------+--------------------+------------+-------------------------+-----------------------+-----------------------+
|src|                 dst|total_departure_delays|total_arrival_delays|flight_count|percent_departure_delayed|percent_arrival_delayed|average_percent_delayed|
+---+--------------------+----------------------+--------------------+------------+-------------------------+-----------------------+-----------------------+
|GRB|      Louisville, KY|                     1|                   1|           1|                    100.0|                  100.0|                  100.0|
|FLG|         Durango, CO|                     1|                   1|           1|                    100.0|                  100.0|                  100.0|
|FAT|           Boise, ID|                     1|                   1|           1|                    100.0|                  100.0|                  100.0|
|ORH|West Palm Beach/P...|                     1|   

In [150]:
# Most and least delayed routes by both arrival and departure delay
total_delays_by_route = graph.edges.groupBy("src", "dst").agg(
    F.sum("ArrDel15").alias("total_arrival_delays"),  # Sum of arrival delays
    F.sum("DepDel15").alias("total_departure_delays"),  # Sum of departure delays
    F.count("*").alias("flight_count")  # Count of flights
).withColumn(
    "total_delays",
    F.col("total_arrival_delays") + F.col("total_departure_delays")
)

# Route with most delays
most_delays_route = total_delays_by_route.orderBy("total_delays", ascending=False).limit(5)

# Route with least delays
least_delays_route = total_delays_by_route.orderBy("total_delays", ascending=True).filter(F.col("flight_count") > 100).limit(5)  # Filtering to avoid routes with very few flights

most_delays_route.show()
least_delays_route.show()


+---+------------+--------------------+----------------------+------------+------------+
|src|         dst|total_arrival_delays|total_departure_delays|flight_count|total_delays|
+---+------------+--------------------+----------------------+------------+------------+
|BOS|New York, NY|              124089|                 98208|      604752|      222297|
|ATL|New York, NY|               90126|                 83699|      338258|      173825|
|DCA|New York, NY|               98717|                 68308|      481891|      167025|
|ORD|New York, NY|               86977|                 79122|      325733|      166099|
|ATL| Chicago, IL|               84595|                 77589|      363053|      162184|
+---+------------+--------------------+----------------------+------------+------------+



+---+--------------+--------------------+----------------------+------------+------------+
|src|           dst|total_arrival_delays|total_departure_delays|flight_count|total_delays|
+---+--------------+--------------------+----------------------+------------+------------+
|FNT|  Muskegon, MI|                   1|                     1|         105|           2|
|MKE|  Portland, OR|                   1|                     2|         155|           3|
|MKK|   Kahului, HI|                   2|                     4|         286|           6|
|SGU|Cedar City, UT|                   3|                     4|         143|           7|
|EYW|Fort Myers, FL|                   4|                     4|         220|           8|
+---+--------------+--------------------+----------------------+------------+------------+



In [161]:
#Most delayed routes by percentage of DEP delays
from pyspark.sql.functions import desc

count_of_flights = 500  # only flights with >500 routes flown

most_dep_delayed_routes = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_departure_delays"),  
    F.count("*").alias("flight_count")  
).withColumn(
    "percent_departure_delayed",
    (F.col("total_departure_delays") / F.col("flight_count")) * 100
).filter(
    F.col("flight_count") >= count_of_flights
).orderBy(
    desc("percent_departure_delayed"),
    desc("flight_count")
).limit(10) 

most_dep_delayed_routes.select("src", "dst", "percent_departure_delayed", "flight_count").show()


+---+-------------------+-------------------------+------------+
|src|                dst|percent_departure_delayed|flight_count|
+---+-------------------+-------------------------+------------+
|JAC|         Newark, NJ|        45.15765765765766|         888|
|BTR|         Newark, NJ|        44.97681607418856|         647|
|SHV|      Las Vegas, NV|        44.94949494949495|         594|
|CMH|  San Francisco, CA|        44.18181818181818|         550|
|EWR|     Fort Wayne, IN|       43.613707165109034|        2568|
|SNA|      Nashville, TN|        43.43675417661098|         838|
|SCK|        Phoenix, AZ|        43.28621908127209|         566|
|ORF|Fort Lauderdale, FL|        43.26375711574953|         527|
|SRQ|         Denver, CO|                    43.04|         625|
|IAD|     Birmingham, AL|       42.988204456094365|         763|
+---+-------------------+-------------------------+------------+



In [163]:
#Most delayed routes by percentage of ARRIVAL delays
from pyspark.sql.functions import desc

count_of_flights = 500  # only flights with >500 routes flown

most_arr_delayed_routes = graph.edges.groupBy("src", "dst").agg(
    F.sum("ArrDel15").alias("total_arr_delays"),  
    F.count("*").alias("flight_count")  
).withColumn(
    "percent_arr_delayed",
    (F.col("total_arr_delays") / F.col("flight_count")) * 100
).filter(
    F.col("flight_count") >= count_of_flights
).orderBy(
    desc("percent_arr_delayed"),
    desc("flight_count")
).limit(10) 

most_arr_delayed_routes.select("src", "dst", "percent_arr_delayed", "flight_count").show()


+---+-------------------+-------------------+------------+
|src|                dst|percent_arr_delayed|flight_count|
+---+-------------------+-------------------+------------+
|BTR|         Newark, NJ|    58.887171561051|         647|
|EWR|    Albuquerque, NM| 50.481430536451164|         727|
|IAD|     Birmingham, AL|  48.62385321100918|         763|
|LGA|    Little Rock, AR|  46.67842445620224|        1701|
|EWR|     Fort Wayne, IN|  45.01557632398754|        2568|
|JAC|         Newark, NJ| 44.932432432432435|         888|
|ORF|Fort Lauderdale, FL| 44.212523719165084|         527|
|SCK|        Phoenix, AZ|  43.46289752650176|         566|
|EWR|    Little Rock, AR| 42.534129692832764|        4688|
|LEX|         Newark, NJ|  42.49602773364148|        6923|
+---+-------------------+-------------------+------------+



In [165]:
#LEAST delayed routes by % of DEPARTURE delays 

count_of_flights = 500  

least_dep_delayed_routes = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_departure_delays"),  
    F.count("*").alias("flight_count")  
).withColumn(
    "percent_departure_delayed",
    (F.col("total_departure_delays") / F.col("flight_count")) * 100
).filter(
    F.col("flight_count") >= count_of_flights
).orderBy(
    asc("percent_departure_delayed"),
    desc("flight_count")
).limit(10) 

least_dep_delayed_routes.select("src", "dst", "percent_departure_delayed", "flight_count").show()


+---+------------------+-------------------------+------------+
|src|               dst|percent_departure_delayed|flight_count|
+---+------------------+-------------------------+------------+
|ITO|          Kona, HI|       1.3731825525040386|        1238|
|IPL|     San Diego, CA|        1.948051948051948|        1078|
|MWH|       Seattle, WA|       2.2222222222222223|         675|
|MFR|        Eugene, OR|       2.5284450063211126|         791|
|VEL|Salt Lake City, UT|       2.6243093922651934|         724|
|XNA|     Cleveland, OH|       2.6548672566371683|        1017|
|MDT|     Cleveland, OH|       2.9394644935972063|       10308|
|SLC|        Vernal, UT|       3.1724137931034484|         725|
|OMA|      Portland, OR|       3.3057851239669422|         847|
|ROW|   Los Angeles, CA|         3.64963503649635|         822|
+---+------------------+-------------------------+------------+



In [ ]:
#LEAST delayed routes by % of DEPARTURE delays 

count_of_flights = 500  

least_dep_delayed_routes = graph.edges.groupBy("src", "dst").agg(
    F.sum("DepDel15").alias("total_dep_delays"),  
    F.count("*").alias("flight_count")  
).withColumn(
    "percent_dep_delayed",
    (F.col("total_dep_delays") / F.col("flight_count")) * 100
).filter(
    F.col("flight_count") >= count_of_flights
).orderBy(
    asc("percent_dep_delayed"),
    desc("flight_count")
).limit(10) 

least_dep_delayed_routes.select("src", "dst", "percent_dep_delayed", "flight_count").show()


In [166]:
#LEAST delayed routes by % of ARRIVAL delays 

count_of_flights = 500  

least_arr_delayed_routes = graph.edges.groupBy("src", "dst").agg(
    F.sum("ArrDel15").alias("total_arr_delays"),  
    F.count("*").alias("flight_count")  
).withColumn(
    "percent_arr_delayed",
    (F.col("total_arr_delays") / F.col("flight_count")) * 100
).filter(
    F.col("flight_count") >= count_of_flights
).orderBy(
    asc("percent_arr_delayed"),
    desc("flight_count")
).limit(10) 

least_arr_delayed_routes.select("src", "dst", "percent_arr_delayed", "flight_count").show()


23/08/16 03:27:29 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 735 for reason Container marked as failed: container_1692138744335_0001_01_000749 on host: hub-hub-msca-bdp-dphub-student-sanchiarora-sw-vd0j.c.msca-bdp-student-ap.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.
23/08/16 03:27:29 WARN org.apache.spark.scheduler.cluster.YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 734 for reason Container marked as failed: container_1692138744335_0001_01_000748 on host: hub-hub-msca-bdp-dphub-student-sanchiarora-sw-vd0j.c.msca-bdp-student-ap.internal. Exit status: -100. Diagnostics: Container released on a *lost* node.


+---+------------------+-------------------+------------+
|src|               dst|percent_arr_delayed|flight_count|
+---+------------------+-------------------+------------+
|ITO|          Kona, HI| 1.2924071082390953|        1238|
|MFR|        Eugene, OR| 2.5284450063211126|         791|
|XNA|     Cleveland, OH|  2.949852507374631|        1017|
|IPL|     San Diego, CA|  3.710575139146568|        1078|
|LGB|        Fresno, CA| 3.7735849056603774|         530|
|MWH|       Seattle, WA|                4.0|         675|
|FAT|       Ontario, CA|  4.105571847507331|         682|
|CNY|Salt Lake City, UT|               4.25|        1200|
|SLC|        Vernal, UT|  4.413793103448276|         725|
|VEL|Salt Lake City, UT|   4.41988950276243|         724|
+---+------------------+-------------------+------------+

